<a href="https://colab.research.google.com/github/yee030/YOLo/blob/main/YOLo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# GitHub에서 YOLOv5 레포지토리를 복제합니다.
!git clone https://github.com/ultralytics/yolov5  # clone

# yolov5 디렉토리로 이동합니다.
%cd yolov5

# requirements.txt 파일에 명시된 필수 라이브러리 및 comet_ml 라이브러리를 설치합니다.
%pip install -qr requirements.txt comet_ml  # install

# PyTorch 및 관련 라이브러리를 초기화하여 사용 가능한지 확인합니다.
import torch
import utils
display = utils.notebook_init()  # 환경 초기화 및 노트북에서 사용 가능한지 확인합니다.


YOLOv5 🚀 v7.0-218-g9e97ac3 Python-3.10.12 torch-2.0.1+cu118 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/107.7 GB disk)


In [ ]:
# YOLOv5로 객체 탐지 수행하는 명령을 실행합니다.
# --weights yolov5s.pt: 사전 훈련된 모델 가중치 파일(yolov5s.pt)을 사용합니다.
# --img 640: 입력 이미지의 크기를 640x640 픽셀로 설정합니다.
# --conf 0.25: 신뢰도 임계값을 0.25로 설정하여 낮은 확률의 예측도 표시합니다.
# --source data/images: 탐지를 수행할 이미지가 있는 디렉토리 경로를 지정합니다.
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images

# 아래 주석은 이미지를 출력하는 코드입니다. (노트북 환경에서 사용)
# display.Image(filename='runs/detect/exp/zidane.jpg', width=600)


detect: weights=['yolov5s.pt'], source=data/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-218-g9e97ac3 Python-3.10.12 torch-2.0.1+cu118 CPU

100% 14.1M/14.1M [00:00<00:00, 91.9MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
image 1/2 /content/yolov5/data/images/bus.jpg: 640x480 4 persons, 1 bus, 398.3ms
image 2/2 /content/yolov5/data/images/zidane.jpg: 384x640 2 persons, 2 ties, 382.4ms
Speed: 2.3ms pre-process, 390.4ms inference, 18.6ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp


In [ ]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d ../datasets && rm tmp.zip  # unzip

100%|██████████| 780M/780M [00:14<00:00, 55.6MB/s]


In [ ]:
# Validate YOLOv5s on COCO val
!python val.py --weights yolov5s.pt --data coco.yaml --img 640 --half

val: data=/content/yolov5/data/coco.yaml, weights=['yolov5s.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-218-g9e97ac3 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
val: Scanning /content/datasets/coco/val2017... 4952 images, 48 backgrounds, 0 corrupt: 100% 5000/5000 [00:03<00:00, 1500.61it/s]
val: New cache created: /content/datasets/coco/val2017.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0% 0/157 [00:01<?, ?it/s]
Traceback (most recent call last):
  File "/content/yolov5/val.py", line 411, in <module>
    main(opt)
  File "/content/yolov5/val.py", line 382, in main
    run(**vars(opt))
  File "/usr/local/lib

In [ ]:
#@title Select YOLOv5 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'ClearML', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'ClearML':
  %pip install -q clearml
  import clearml; clearml.browser_login()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)


In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache

In [ ]:
# YOLOv5 PyTorch HUB Inference (DetectionModels only)
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True)  # yolov5n - yolov5x6 or custom
im = 'https://ultralytics.com/images/zidane.jpg'  # file, Path, PIL.Image, OpenCV, nparray, list
results = model(im)  # inference
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.